In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import time

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from datagen.oversampler import oversample   # LLM oversample
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)   # 5-fold-cross validation

In [ ]:
data_list = [14, 6]

# 1.SMOTE-5CV (48% x 5 times)

In [ ]:
time_list = []
# Data Generation 
for i in [6]:
    df = pd.read_csv('data_newest/ds'+ str(i) +'_new.csv')
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    
    # Make major class as '0' and minor class as '1'
    MAJOR = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() == max(df.iloc[:,-1].value_counts())].index[0] # Moj Label
    minor = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() != max(df.iloc[:,-1].value_counts())].index[0] # min Label    
    df.iloc[:,-1] = df.iloc[:,-1].replace(MAJOR, -100)
    df.iloc[:,-1] = df.iloc[:,-1].replace(minor, 1)
    df.iloc[:,-1] = df.iloc[:,-1].replace(-100, 0)
    df.rename(columns={df.columns[-1]:'NEW_LABEL'}, inplace=True)
    print('<Modified Class>\n', df.iloc[:,-1].value_counts())
    print('<Imabalance ratio>\n', "1:{: .3f}".format(df.iloc[:,-1].value_counts()[1]/df.iloc[:,-1].value_counts()[0]))
        
    ##################### Validation:Test = 70:30 #######################
    df_val, df_test = train_test_split(df, test_size=0.3, random_state=100, stratify=df.iloc[:,-1])
    X = df_val.iloc[:, :-1]
    y = df_val.iloc[:, -1]
    
    ##################### For Validation Set #######################
    Strategy = [0.2, 0.3, 0.4, 0.5]
    ind = int((y.value_counts()[1]/y.value_counts()[0])//0.2)
    min_strategy = Strategy[ind]
    if i == 33:
        min_strategy = Strategy[1]
    print("<min_strategy>:",min_strategy)   
    
    for j in range(len(Strategy)):
        print("==========", "SMOTE_{}".format(Strategy[j]), "==========") 
        over = SMOTE(sampling_strategy=Strategy[j], random_state=0)
        if min_strategy > Strategy[j]:
            continue         
        else:     
            n_iter=0
            for train_index, val_index in skf.split(X, y):
                n_iter += 1
                print("=======", "{}th-cv".format(n_iter), "=======")
                X_train = X.iloc[train_index]
                y_train= y.iloc[train_index]
                if n_iter == 1:
                    print(list(y_train).count(0), list(y_train).count(1), len(y_train))
                df_train = pd.concat([X_train, y_train], axis=1)
                minority_data = df_train[df_train["NEW_LABEL"] == 1]
                print("Nedeed Samples:",int((len(df_train)-len(minority_data))*Strategy[j]-len(minority_data)))
                
                # Resmapling
                start = time.time()
                X_train, y_train = over.fit_resample(X_train, y_train)
                end = time.time()
                time_list.append(end-start)
                
                if n_iter == 1:
                    print(list(y_train).count(0), list(y_train).count(1), len(y_train))
                over_df = pd.concat([X_train, y_train], axis=1)
                break

In [ ]:
# time_sm_14 = time_list
time_sm_14

In [ ]:
# time_sm_6 = time_list
time_sm_6

# 3.LLM-5CV (48% x 5 times)

In [ ]:
time_list = []
# Data Generation
for i in [6]:
    df = pd.read_csv('data_newest/ds'+ str(i) +'_new.csv')
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    
    # Make major class as '0' and minor class as '1'
    MAJOR = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() == max(df.iloc[:,-1].value_counts())].index[0] # Moj Label
    minor = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() != max(df.iloc[:,-1].value_counts())].index[0] # min Label    
    df.iloc[:,-1] = df.iloc[:,-1].replace(MAJOR, -100)
    df.iloc[:,-1] = df.iloc[:,-1].replace(minor, 1)
    df.iloc[:,-1] = df.iloc[:,-1].replace(-100, 0)
    df.rename(columns={df.columns[-1]:'NEW_LABEL'}, inplace=True)
    print('<Modified Class>\n', df.iloc[:,-1].value_counts())
    print('<Imabalance ratio>\n', "1:{: .2f}".format(df.iloc[:,-1].value_counts()[1]/df.iloc[:,-1].value_counts()[0]))
        
    ##################### Validation:Test = 70:30 #######################
    df_val, df_test = train_test_split(df, test_size=0.3, random_state=100, stratify=df.iloc[:,-1])
    X = df_val.iloc[:, :-1]
    y = df_val.iloc[:, -1]
    
    ##################### For Validation Set #######################
    Strategy = [0.2, 0.3, 0.4, 0.5]
    ind = int((y.value_counts()[1]/y.value_counts()[0])//0.2)
    min_strategy = Strategy[ind]
    if i == 33:
        min_strategy = Strategy[1]
    print("<min_strategy>:",min_strategy)   
    
    for h in range(len(Strategy)):
        print("==========", "LLM_{}".format(Strategy[h]), "==========")       
        if min_strategy > Strategy[h]:
            continue
        else:     
            n_iter=0
            for train_index, val_index in skf.split(X, y):
                n_iter += 1
                print("=======", "{}th-cv".format(n_iter), "=======")
                X_train = X.iloc[train_index]
                y_train= y.iloc[train_index]
                if n_iter == 1:
                    print(list(y_train).count(0), list(y_train).count(1), len(y_train))
                # Resmapling
                df_train = pd.concat([X_train, y_train], axis=1)
                minority_data = df_train[df_train["NEW_LABEL"] == 1]
                print("Nedeed Samples:",int((len(df_train)-len(minority_data))*Strategy[h]-len(minority_data)))
                
                
                try:
                    start = time.time()
                    new_data = oversample(minority_data, 
                                          int((len(df_train)-len(minority_data))*Strategy[h]-len(minority_data)), # number of generated data samples
                                          "NEW_LABEL",
                                          1, # minor class
                                          single_desc=True, single_vars=True
                                          )
                    end = time.time()
                    time_list.append(end-start)
                except:
                    continue
                
                    
                over_df = pd.concat([df_train, new_data], axis=0)
                if h == 0:
                    over_df.to_csv("data_newest/LLM_over/ds{}_L_{}_{}th_time.csv".format(i, Strategy[h], n_iter), index=False)
                break

In [ ]:
# time_lm_14 = time_list
time_lm_14

In [ ]:
# time_lm_6 = time_list
time_lm_6

# 5. LLM + SMOTE - CV (48% x 5 times)

In [ ]:
time_list = [0]
# Data Generation 
for i in [6]:
    df = pd.read_csv('data_newest/ds'+ str(i) +'_new.csv')
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    
    # Make major class as '0' and minor class as '1'
    MAJOR = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() == max(df.iloc[:,-1].value_counts())].index[0] # Moj Label
    minor = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() != max(df.iloc[:,-1].value_counts())].index[0] # min Label    
    df.iloc[:,-1] = df.iloc[:,-1].replace(MAJOR, -100)
    df.iloc[:,-1] = df.iloc[:,-1].replace(minor, 1)
    df.iloc[:,-1] = df.iloc[:,-1].replace(-100, 0)
    df.rename(columns={df.columns[-1]:'NEW_LABEL'}, inplace=True)
    print('<Modified Class>\n', df.iloc[:,-1].value_counts())
    print('<Imabalance ratio>\n', "1:{: .2f}".format(df.iloc[:,-1].value_counts()[1]/df.iloc[:,-1].value_counts()[0]))
        
    ##################### Validation:Test = 70:30 #######################
    df_val, df_test = train_test_split(df, test_size=0.3, random_state=100, stratify=df.iloc[:,-1])
    X = df_val.iloc[:, :-1]
    y = df_val.iloc[:, -1]
    print(list(y).count(0), list(y).count(1), len(y))
    print(list(y).count(0)*0.8, list(y).count(1)*0.8, len(y)*0.8)
    ##################### For Validation Set #######################
    Strategy = [0.2, 0.3, 0.4, 0.5]
    ind = int((y.value_counts()[1]/y.value_counts()[0])//0.2)
    min_strategy = Strategy[ind]   
    adj_strategy = Strategy[ind+1]  # original min_strategy-> LLM oversmaple, so SMOTE is used from the next 
    if i == 33:
        min_strategy = Strategy[1]
        adj_strategy = Strategy[2]
    print("<min_strategy>:",min_strategy)   
    
    for j in range(len(Strategy)):
        print("==========", "LLM_SMOTE_{}".format(Strategy[j]), "==========") 
        over = SMOTE(sampling_strategy=Strategy[j], random_state=0)
        if adj_strategy > Strategy[j]:
            continue
        else:
            for n_iter in range(1,6):
                print("=======", "{}th-cv".format(n_iter), "=======")
                df_lm = pd.read_csv(r'LLM_over/'
                                +'ds'+str(i)+'_L_'+str(min_strategy)+'_'+str(n_iter)+'th.csv')
                print(str(i), str(min_strategy), str(n_iter))
                df_lm = df_lm.replace('False', False)  # sometimes False happen
                df_lm = df_lm.replace('FALSE', False)  # sometimes False happen
                df_lm = df_lm.fillna(0)   # sometime NAN happen
                X_lm = df_lm.iloc[:, :-1]
                y_lm = df_lm.iloc[:, -1]
                if n_iter == 1:
                    print(list(y_lm).count(0), list(y_lm).count(1), len(y_lm)) 
                minority_data = df_lm[df_lm["NEW_LABEL"] == 1]
                print(len(df_lm), len(minority_data), Strategy[j])
                print("Nedeed Samples:",int((len(df_lm)-len(minority_data))*Strategy[j]-len(minority_data)))
                
                # Resmapling
                start = time.time()
                X_lm, y_lm = over.fit_resample(X_lm, y_lm)
                end = time.time()
                time_list.append(end-start)
                
                if n_iter == 1:
                    print(list(y_lm).count(0), list(y_lm).count(1), len(y_lm))
                over_df = pd.concat([X_lm, y_lm], axis=1)
                break

In [ ]:
# time_ls_14 = []
# for i in range(len(time_list)):
#     time_ls_14.append(time_list[i] + time_lm_14[0])
time_ls_14

In [ ]:
# time_ls_6 = []
# for i in range(len(time_list)):
#     time_ls_6.append(time_list[i] + time_lm_6[0])
time_ls_6